In [1]:
import coax
import gym
import haiku as hk
import jax
import jax.numpy as jnp
import optax
from coax.value_losses import mse
from RPPRegularizer import RPPRegularizer

import matplotlib.pyplot as plt

# the name of this script
name = 'a2c'

# the cart-pole MDP
# env = gym.make('CartPole-v0')
env = gym.make("rpp_gym:InclinedCartpole-v0")
env.alpha = 0.2

In [2]:
env = coax.wrappers.TrainMonitor(env, name=name, tensorboard_dir=f"./data/tensorboard/{name}")

In [3]:
# env.spec.max_episode_steps = 200
# env.spec.reward_threshold = 195.0

In [4]:
from emlp import T, Scalar
from emlp.groups import SO, S, O, Trivial,Z
import emlp.nn.haiku as ehk
from emlp.reps import Rep
from emlp.nn import gated,gate_indices,uniform_rep
from math import prod
from representations import PseudoScalar
from mixed_emlp_haiku import MixedEMLP

## Trivial
# group=Trivial(2)
# rep_in = T(0)*prod(env.observation_space.shape)
# rep_out = T(0)*env.action_space.n#prod(env.action_space.shape)

## Reflection
group=Z(2)
rep_in = PseudoScalar()*prod(env.observation_space.shape)
rep_out = T(1)#*env.action_space.n#prod(env.action_space.shape)

ch = 100
num_layers = 3


# nn_pi = ehk.EMLP(rep_in,rep_out,group,ch=ch,num_layers=num_layers)
# nn_v = ehk.EMLP(rep_in,T(0),group,ch=ch,num_layers=num_layers)

# nn_pi = ehk.MLP(rep_in,rep_out(group),group,ch=ch,num_layers=num_layers)
# nn_v = ehk.MLP(rep_in,T(0),group,ch=ch,num_layers=num_layers)

nn_pi = MixedEMLP(rep_in,rep_out(group),group,ch=ch,num_layers=num_layers)
nn_v = MixedEMLP(rep_in,T(0),group,ch=ch,num_layers=num_layers)


def func_pi(S, is_training):
    return {'logits': nn_pi(S)}


def func_v(S, is_training):
    return nn_v(S).reshape(-1)



# def func_pi(S, is_training):
#     logits = hk.Sequential((
#         hk.Linear(16), jax.nn.relu,
#         hk.Linear(16), jax.nn.relu,
#         hk.Linear(16), jax.nn.relu,
#         hk.Linear(env.action_space.n, w_init=jnp.zeros)
#     ))
#     return {'logits': logits(S)}



# def func_v(S, is_training):
#     value = hk.Sequential((
#         hk.Linear(32), jax.nn.relu,
#         hk.Linear(32), jax.nn.relu,
#         hk.Linear(32), jax.nn.relu,
#         hk.Linear(32), jax.nn.relu,
#         hk.Linear(32), jax.nn.relu,
#         hk.Linear(1, w_init=jnp.zeros), jnp.ravel
#     ))
#     return value(S)


[a2c|root|INFO] Initing EMLP
[a2c|root|INFO] Linear W components:400 rep:96P+48P⊗V+20P⊗V²+8P⊗V³+4P⊗V⁴
[a2c|root|INFO] P cache miss
[a2c|root|INFO] Solving basis for P, for G=Z(2)
[a2c|root|INFO] P⊗V cache miss
[a2c|root|INFO] Solving basis for P⊗V, for G=Z(2)
[a2c|root|INFO] P⊗V² cache miss
[a2c|root|INFO] Solving basis for P⊗V², for G=Z(2)
[a2c|root|INFO] P⊗V³ cache miss
[a2c|root|INFO] Solving basis for P⊗V³, for G=Z(2)
[a2c|root|INFO] P⊗V⁴ cache miss
[a2c|root|INFO] Solving basis for P⊗V⁴, for G=Z(2)
[a2c|root|INFO] V cache miss
[a2c|root|INFO] Solving basis for V, for G=Z(2)
[a2c|root|INFO] V² cache miss
[a2c|root|INFO] Solving basis for V², for G=Z(2)
[a2c|root|INFO] V³ cache miss
[a2c|root|INFO] Solving basis for V³, for G=Z(2)
[a2c|root|INFO] V⁴ cache miss
[a2c|root|INFO] Solving basis for V⁴, for G=Z(2)
[a2c|root|INFO] Linear W components:10000 rep:576V⁰+576V+384V²+216V³+121V⁴+44V⁵+14V⁶+4V⁷+V⁸
[a2c|root|INFO] V⁵ cache miss
[a2c|root|INFO] Solving basis for V⁵, for G=Z(2)
[a2c|r

In [5]:
# these optimizers collect batches of grads before applying updates
optimizer_v = optax.chain(optax.apply_every(k=32), optax.adam(0.002))
optimizer_pi = optax.chain(optax.apply_every(k=32), optax.adam(0.001))


# value function and its derived policy
v = coax.V(func_v, env)
pi = coax.Policy(func_pi, env)

In [6]:
%pdb

Automatic pdb calling has been turned ON


In [7]:
# experience tracer
pi_regularizer = RPPRegularizer(pi, basic_wd=1000., equiv_wd=0.)
# v_regularizer = RPPRegularizer(v, basic_wd=1e-5, equiv_wd=0.)
tracer = coax.reward_tracing.NStep(n=1, gamma=0.9)

# updaters
vanilla_pg = coax.policy_objectives.VanillaPG(pi, optimizer=optimizer_pi,
                                              regularizer=pi_regularizer)
simple_td = coax.td_learning.SimpleTD(v, loss_function=mse, optimizer=optimizer_v,
                                     policy_regularizer=pi_regularizer)

epoch_rewards = []

# train
for ep in range(100):
    s = env.reset()
    er = 0
    for t in range(env.spec.max_episode_steps):
        a = pi(s)
        s_next, r, done, info = env.step(a)
        
        if done and (t == env.spec.max_episode_steps - 1):
            r = 1 / (1 - tracer.gamma)
        er+=r
        tracer.add(s, a, r, done)
        while tracer:
            transition_batch = tracer.pop()
            metrics_v, td_error = simple_td.update(transition_batch, return_td_error=True)
            metrics_pi = vanilla_pg.update(transition_batch, td_error)
            env.record_metrics(metrics_v)
            env.record_metrics(metrics_pi)

        if done:
            break

        s = s_next
    
    print("Epoch reward",er)
    epoch_rewards.append(er)
    # early stopping
    if env.avg_G > env.spec.reward_threshold:
        break


# run env one more time to render
#coax.utils.generate_gif(env, policy=pi, filepath=f"./data/{name}.gif", duration=25)

NameError: name 'test' is not defined

> /home/greg_b/residual-pathway-priors/RL/RPPRegularizer.py(25)function()
     23         def function(dist_params, equiv_wd, basic_wd):
     24 #             print(dist_params)
---> 25             if test:
     26                 print(temperatjflkdasjfklas)
     27             equiv_l2 = 0.0



ipdb>  dist_params


{'logits': Traced<ShapedArray(float32[1,2])>with<DynamicJaxprTrace(level=0/3)>}


ipdb>  u


...

ipdb>  u


> /home/greg_b/miniconda3/envs/rpp/lib/python3.8/site-packages/jax/core.py(1405)process()
   1403 
   1404   def process(self, trace, fun, tracers, params):
-> 1405     return trace.process_call(self, fun, tracers, params)
   1406 
   1407   def post_process(self, trace, out_tracers, params):



ipdb>  u


> /home/greg_b/miniconda3/envs/rpp/lib/python3.8/site-packages/jax/core.py(1393)call_bind()
   1391   tracers = map(top_trace.full_raise, args)
   1392   with maybe_new_sublevel(top_trace):
-> 1393     outs = primitive.process(top_trace, fun, tracers, params)
   1394   return map(full_lower, apply_todos(env_trace_todo(), outs))
   1395 



ipdb>  q


In [ ]:
vanilla_pg

In [ ]:

metrics_pi

In [ ]:
equiv_l2 = 0.0
basic_l2 = 0.0
for k1, v1 in pi.params.items():
    if "bi" not in k1:
        for k2, v2 in v1.items():
    #         if "bi" not in k2:
    #             print(k2)
            if k2.endswith("w_basic"):
                basic_l2 += (v2 ** 2).sum()
            elif k2.endswith("w"):
                equiv_l2 += (v2 ** 2).sum()
print("Eq l2 = ", equiv_l2)
print("Basic l2 = ", basic_l2)

In [ ]:
# coax.utils.dump(pi.params, "./emlp_pi_params.lz4")
# coax.utils.dump(v.params, "./emlp_v_params.lz4")